In [6]:
import pandas as pd
df = pd.read_excel("/home/elson/topk3_minilm.xlsx", engine='openpyxl')



In [24]:
df.head()

,Unnamed: 0,folder,filename,claim,label,url,GOLD EXPLANATION,CATEGORY,gemini_label,gemini_explanation,top_1_minilm_ce,top_2_minilm_ce,top_3_minilm_ce,claim_tokens,hyp1_tokens,hyp2_tokens,hyp3_tokens
0,0,/content/drive/MyDrive/images/myths on urticaria,mythsonurticaria1.jpeg,Eating chocolate will cause acne.,SUPPORTED,https://www.jaad.org/article/S0190-9622(16)013...,The chocolate consumption group had a statisti...,Skin,REFUTED,There is no scientific evidence to support the...,The link of chocolate to acne vulgaris was rep...,A 2021 systematic review of 53 studies (11 int...,Fig 2 demonstrates that the chocolate consumpt...,"[eating, chocolate, will, cause, acne]","[the, link, of, chocolate, to, acne, vulgaris,...","[a, 2021, systematic, review, of, 53, studies,...","[fig, 2, demonstrates, that, the, chocolate, c..."
1,1,/content/drive/MyDrive/images/myths on urticaria,mythsonurticaria1.jpeg,You can get a cold from being in the rain.,SUPPORTED,https://pubmed.ncbi.nlm.nih.gov/17705968/,Exposure to cold has often been associated wi...,General Health,REFUTED,"The common cold is caused by viruses, not by b...",The data available suggest that exposure to co...,This mechanism can explain why a person who ex...,"As a general observation, wet hair in cold wea...","[you, can, get, a, cold, from, being, in, the,...","[the, data, available, suggest, that, exposure...","[this, mechanism, can, explain, why, a, person...","[as, a, general, observation, wet, hair, in, c..."
2,2,/content/drive/MyDrive/images/myths on urticaria,mythsonurticaria1.jpeg,Stress can cause acne.,SUPPORTED,https://medicaljournalssweden.se/actadv/articl...,"Based on this study, increased stress does not...",Skin,SUPPORTED,There is evidence to suggest that stress can t...,This finding provides physiological support to...,The impact of pyschological stress on acne.,Both active acne and post-inflammatory hyperpi...,"[stress, can, cause, acne]","[this, finding, provides, physiological, suppo...","[the, impact, of, pyschological, stress, on, a...","[both, active, acne, and, postinflammatory, hy..."
3,3,/content/drive/MyDrive/images/myths on urticaria,mythsonurticaria1.jpeg,You can prevent acne by washing your face more...,NOT ENOUGH INFORMATION,https://www.tandfonline.com/doi/full/10.1080/0...,Washing and over-the-counter cleansers are com...,Skin,REFUTED,Washing your face more often does not prevent ...,Purpose: Washing and over-the-counter cleanser...,Patients can also be advised to pat dry their ...,Treatment of acne should be started early to p...,"[you, can, prevent, acne, by, washing, your, f...","[purpose, washing, and, overthecounter, cleans...","[patients, can, also, be, advised, to, pat, dr...","[treatment, of, acne, should, be, started, ear..."
4,4,/content/drive/MyDrive/images/myths on vascula...,mythsonvascularsurgery3.jpeg,Varicose veins are caused by standing too much.,SUPPORTED,https://www.sjweh.fi/article/562,"For men working mostly in a standing position,...",Vascular,NOT ENOUGH INFORMATION,While standing for long periods of time can co...,Varicose veins are superficial veins in the su...,Varicose veins are caused by poorly functionin...,Varicose veins are caused by poorly functionin...,"[varicose, veins, are, caused, by, standing, t...","[varicose, veins, are, superficial, veins, in,...","[varicose, veins, are, caused, by, poorly, fun...","[varicose, veins, are, caused, by, poorly, fun..."


In [25]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/elson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
import string

def process_text_lowercase(sentence):
    # Import string module inside the function to avoid any conflicts
    import string
    sentence = str(sentence)
    tokens = sentence.split()
    tokens = [''.join(char for char in word if char not in string.punctuation).lower() for word in tokens]
    return tokens

df['claim_tokens'] = df['claim'].apply(process_text_lowercase)


In [21]:
df['hyp1_tokens'] = df['top_1_minilm_ce'].apply(process_text_lowercase)

In [22]:
df['hyp2_tokens'] = df['top_2_minilm_ce'].apply(process_text_lowercase)

In [23]:
df['hyp3_tokens'] = df['top_3_minilm_ce'].apply(process_text_lowercase)

In [37]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

# Make sure to download the required NLTK model for tokenization
nltk.download('punkt')

# Define the heuristic checks directly operating on token lists
def check_lexical_overlap(premise_tokens, hypothesis_tokens):
    return all(token in premise_tokens for token in hypothesis_tokens)

def check_subsequence(premise_tokens, hypothesis_tokens):
    it = iter(premise_tokens)
    return all(token in it for token in hypothesis_tokens)

def check_constituent(premise_tokens, hypothesis_tokens):
    premise_str = ' '.join(premise_tokens)
    hypothesis_str = ' '.join(hypothesis_tokens)
    return hypothesis_str in premise_str

# Function to apply the heuristic checks for each hypothesis and add the results as new columns
def apply_heuristic_checks(df):
    for i in range(1, 4):
        hyp_col = f'hyp{i}_tokens'  # Column name for hypothesis tokens
        
        # Applying each heuristic check
        df[f'heuristic_{i}_lexical_overlap'] = df.apply(
            lambda row: "Lexical Overlap" if check_lexical_overlap(row['claim_tokens'], row[hyp_col]) else '', axis=1)
        df[f'heuristic_{i}_subsequence'] = df.apply(
            lambda row: "Subsequence" if check_subsequence(row['claim_tokens'], row[hyp_col]) else '', axis=1)
        df[f'heuristic_{i}_constituent'] = df.apply(
            lambda row: "Constituent" if check_constituent(row['claim_tokens'], row[hyp_col]) else '', axis=1)
    
    return df

# Assuming 'df' is your DataFrame, apply the heuristic checks
df = apply_heuristic_checks(df)

# Now, 'df' includes new columns for the heuristics applied to each pair of claim and hypotheses


[nltk_data] Downloading package punkt to /home/elson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [39]:
df.head()

,Unnamed: 0,folder,filename,claim,label,url,GOLD EXPLANATION,CATEGORY,gemini_label,gemini_explanation,...,hyp3_tokens,heuristic_1_lexical_overlap,heuristic_1_subsequence,heuristic_1_constituent,heuristic_2_lexical_overlap,heuristic_2_subsequence,heuristic_2_constituent,heuristic_3_lexical_overlap,heuristic_3_subsequence,heuristic_3_constituent
0,0,/content/drive/MyDrive/images/myths on urticaria,mythsonurticaria1.jpeg,Eating chocolate will cause acne.,SUPPORTED,https://www.jaad.org/article/S0190-9622(16)013...,The chocolate consumption group had a statisti...,Skin,REFUTED,There is no scientific evidence to support the...,...,"[fig, 2, demonstrates, that, the, chocolate, c...",,,,,,,,,
1,1,/content/drive/MyDrive/images/myths on urticaria,mythsonurticaria1.jpeg,You can get a cold from being in the rain.,SUPPORTED,https://pubmed.ncbi.nlm.nih.gov/17705968/,Exposure to cold has often been associated wi...,General Health,REFUTED,"The common cold is caused by viruses, not by b...",...,"[as, a, general, observation, wet, hair, in, c...",,,,,,,,,
2,2,/content/drive/MyDrive/images/myths on urticaria,mythsonurticaria1.jpeg,Stress can cause acne.,SUPPORTED,https://medicaljournalssweden.se/actadv/articl...,"Based on this study, increased stress does not...",Skin,SUPPORTED,There is evidence to suggest that stress can t...,...,"[both, active, acne, and, postinflammatory, hy...",,,,,,,,,
3,3,/content/drive/MyDrive/images/myths on urticaria,mythsonurticaria1.jpeg,You can prevent acne by washing your face more...,NOT ENOUGH INFORMATION,https://www.tandfonline.com/doi/full/10.1080/0...,Washing and over-the-counter cleansers are com...,Skin,REFUTED,Washing your face more often does not prevent ...,...,"[treatment, of, acne, should, be, started, ear...",,,,,,,,,
4,4,/content/drive/MyDrive/images/myths on vascula...,mythsonvascularsurgery3.jpeg,Varicose veins are caused by standing too much.,SUPPORTED,https://www.sjweh.fi/article/562,"For men working mostly in a standing position,...",Vascular,NOT ENOUGH INFORMATION,While standing for long periods of time can co...,...,"[varicose, veins, are, caused, by, poorly, fun...",,,,,,,,,


In [40]:
df.to_excel('/home/elson/heuristics.xlsx')
